
## Comprendiendo GridSearch con un modelo individual

Antes de ejecutar todos los modelos, veamos **cómo funciona GridSearch** con un solo modelo, la **Regresión Logística**.  
Esto nos ayudará a entender qué hace exactamente el proceso y cómo elige los mejores hiperparámetros.


In [ ]:
# Entrenamiento simple de una Regresión Logística

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Crear y entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Predicción y evaluación
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")



### Ajustando Hiperparámetros Manualmente

Si no especificamos hiperparámetros, Scikit Learn usa valores por defecto.  
Podemos definirlos manualmente para ajustar el comportamiento del modelo:


In [ ]:
# Regresión Logística con hiperparámetros definidos manualmente

model = LogisticRegression(
    C=0.5,                  # Regularización
    penalty='l2',           # Tipo de penalización
    solver='lbfgs',         # Algoritmo de optimización
    max_iter=200,           # Iteraciones máximas
    class_weight='balanced' # Ajuste de clases
)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predicción y evaluación
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")



###  ¿Qué hace GridSearch?

GridSearch prueba **todas las combinaciones posibles de hiperparámetros** que especifiquemos.  
Por ejemplo, para la Regresión Logística podemos definir el siguiente espacio de búsqueda:


In [ ]:
from sklearn.model_selection import GridSearchCV

parametros = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 500, 1000]
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=parametros,
    cv=5,
    scoring='accuracy',
    verbose=0,
    n_jobs=-1
)

# Ajustar el modelo
grid_search.fit(X_train, y_train)

# Evaluar resultados
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Mejor combinación de hiperparámetros: {grid_search.best_params_}")
print(f"Precisión del mejor modelo: {accuracy:.2f}")



 Con esto comprendemos cómo **GridSearch** busca la mejor combinación de hiperparámetros.  
A continuación, aplicaremos el mismo principio a **todos los modelos** de clasificación en el conjunto Titanic.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas
from sklearn.metrics import accuracy_score

# Guardado de modelos
import pickle


In [ ]:
# Cargar el dataset procesado
df = pd.read_csv('./data/titanic_procesado.csv')
df.head()


In [ ]:
# Separar variables independientes (X) y dependiente (y)
X = df.drop(['Survived'], axis=1)
y = df['Survived']

# División entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir a NumPy arrays
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

print("Tamaños:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


In [ ]:
modelos = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {'model__C': [0.1], 'model__max_iter': [1000]}
    },
    'Support Vector Classifier': {
        'model': SVC(),
        'params': {'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'model__C': [0.1, 1, 10]}
    },
    'Decision Tree Classifier': {
        'model': DecisionTreeClassifier(),
        'params': {'model__splitter': ['best', 'random'], 'model__max_depth': [None, 1, 2, 3, 4]}
    },
    'Random Forest Classifier': {
        'model': RandomForestClassifier(),
        'params': {'model__n_estimators': [10, 100], 'model__max_depth': [None, 1, 2, 3, 4], 'model__max_features': ['auto', 'sqrt', 'log2']}
    },
    'Gradient Boosting Classifier': {
        'model': GradientBoostingClassifier(),
        'params': {'model__n_estimators': [10, 100], 'model__max_depth': [None, 1, 2, 3, 4]}
    },
    'AdaBoost Classifier': {
        'model': AdaBoostClassifier(),
        'params': {'model__n_estimators': [10, 100]}
    },
    'K-Nearest Neighbors Classifier': {
        'model': KNeighborsClassifier(),
        'params': {'model__n_neighbors': [3, 5, 7]}
    },
    'XGBoost Classifier': {
        'model': XGBClassifier(),
        'params': {'model__n_estimators': [10, 100], 'model__max_depth': [None, 1, 2, 3]}
    },
    'LGBM Classifier': {
        'model': LGBMClassifier(),
        'params': {'model__n_estimators': [10, 100], 'model__max_depth': [None, 1, 2, 3], 'model__learning_rate': [0.1, 0.2, 0.3], 'model__verbose': [-1]}
    },
    'GaussianNB': {
        'model': GaussianNB(),
        'params': {}
    },
    'Naive Bayes Classifier': {
        'model': BernoulliNB(),
        'params': {'model__alpha': [0.1, 1.0, 10.0]}
    }
}


In [ ]:
resultados = []

for nombre, config in modelos.items():
    print(f"Entrenando {nombre}...")
    pipe = Pipeline([('model', config['model'])])
    grid = GridSearchCV(pipe, config['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    resultados.append((nombre, acc, grid.best_params_))

# Mostrar resultados
resultados_df = pd.DataFrame(resultados, columns=['Modelo', 'Precisión', 'Mejores Parámetros']).sort_values(by='Precisión', ascending=False)
resultados_df.reset_index(drop=True, inplace=True)
resultados_df


In [ ]:
# Guardar el mejor modelo
mejor_modelo = resultados_df.iloc[0]['Modelo']
print(f"Mejor modelo: {mejor_modelo}")

modelo_final = modelos[mejor_modelo]['model']
modelo_final.fit(X_train, y_train)

with open('./data/mejor_modelo.pkl', 'wb') as f:
    pickle.dump(modelo_final, f)

print("Modelo guardado como ./data/mejor_modelo.pkl")


In [ ]:
# Ejecutar el ajuste de los modelos con GridSearchCV

# Definir los modelos y sus respectivos hiperparámetros para GridSearch
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'splitter': ['best', 'random'],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4],
            'max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {'n_estimators': [10, 100]}
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {'n_neighbors': [3, 5, 7]}
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {'n_estimators': [10, 100], 'max_depth': [None, 1, 2, 3]}
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {'n_estimators': [10, 100], 'max_depth': [None, 1, 2, 3], 'learning_rate': [0.1, 0.2, 0.3], 'verbose': [-1]}
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {'alpha': [0.1, 1.0, 10.0]}
    }
}

# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
    print(f"Entrenando {nombre}...")
    grid_search = GridSearchCV(
        estimator=info_modelo['modelo'],
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    precision = accuracy_score(y_test, y_pred)

    puntajes_modelos.append({'Modelo': nombre, 'Precisión': precision})
    estimadores[nombre] = grid_search.best_estimator_

    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_

# Convertir resultados a DataFrame y mostrar métricas
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

# Guardar el mejor modelo encontrado
with open('./data/mejor_modelo_grid.pkl', 'wb') as f:
    pickle.dump(mejor_estimador, f)

print("✅ Mejor modelo guardado como ./data/mejor_modelo_grid.pkl")



## Inferencia y Guardado del Modelo

Llegamos al momento final: **usar nuestro modelo entrenado para predecir nuevos datos**.  
Este proceso se llama *inferencia*, y consiste en aplicar el modelo sobre información nueva que **no vio durante el entrenamiento**.

Podemos alimentar el modelo con datos de nuevos pasajeros y predecir si sobrevivirían o no.


In [ ]:
# Veamos un ejemplo de dato del conjunto de entrenamiento
print("Primer registro de X_train:", X_train[0])
print("Etiqueta correspondiente de y_train:", y_train[0])


In [ ]:
# Crear un nuevo array con los mismos valores de ejemplo
nuevos_datos = np.array([0,1,0.6159084,0,0,0.55547282,1]).reshape(1,-1)

# Realizar la predicción
prediccion = mejor_estimador.predict(nuevos_datos)
print("Predicción del modelo:", prediccion)

if prediccion[0] == 1:
    print("✅ El modelo predice que el pasajero sobrevivió.")
else:
    print("❌ El modelo predice que el pasajero no sobrevivió.")



###  Guardar el Modelo con Pickle

Para reutilizar nuestro modelo en producción (por ejemplo, dentro de una API o app web),  
debemos guardarlo. Usaremos **Pickle**, que permite serializar objetos de Python en archivos.


In [ ]:
import pickle

# Guardar el modelo entrenado
with open('modelo.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)

print("✅ Modelo guardado correctamente como 'modelo.pkl'")
